# Simulating Process Flow

# Stages of Manufacturing

## 1. [Raw Material Inventory Store](#first-bullet)

## 2. [Classifier](#second-bullet)

## 3. [Pre-Finish Inventory Store](#third-bullet)

## 4. [Pre-Finish Operation](#fourth-bullet)

## 5. [Pack Inventory Store](#fifth-bullet)

## 6. [Packaging](#sixth-bullet)

![process_flow_diagram](process_flow_diagram.svg)

# TODO:
- **RMI Store**
    - order_drums(): sort drums as dictated by work orders 
- **Classifier**
    - check_pfi_bins(): ensure >5 empty bins available in PFI Store
- **PFI Store**
    - assign_drums(): ensure that a given drum maintains the same color & size through a given work order
- **PFO Cell**
    - \_\_repr\_\_(): code proper representation of PFO Cell
    - assign_tanks(): determine number of tanks for a given site and initialize them
- **PI Cell**
    - assign_drums(): 
- **Packaging Cell**
- **Work Orders**
- Finish defining packing cell:
    - Assigning machines & properties
    - Defining bagging & boxing machines
    - Determining process rates
- ~~Replace `location` with `plant_id`; update CSVs accordingly~~
- Define how Work Orders are loaded and processed
- Code the overarching simulation process
- Refactoring:
    - Define super/subclass relationship for drums
    - Define super/subclass relationship for workcells?
- Thoroughly comment code cells
- Add markdown to illustrate important concepts; break up code cells as necessary
- Add visualizations
- Flowchart Edit Notes:
    - Work orders dictate RMI Store drum order as well as PI Store order.
    - Number and label each work cell for reference in the document.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [106]:
class facility:
    def __init__(self, plant_id):
        self.plant_id = plant_id
        print("Building {0} Facility Details...".format(plant_id))
        
        equipment = pd.read_csv('equipment.csv')
        equipment = equipment[equipment.facility == plant_id]
        
        # Get relevant equipment details for a given location
        self.rmi_drums = equipment.rmi_drums.values[0]
        self.pfi_drums = equipment.pfi_drums.values[0]
        self.pfo_tanks = equipment.pfo_tanks.values[0]
        self.pi_drums = equipment.pi_drums.values[0]
        self.bagging_machines = equipment.bagging_machines.values[0]
        self.boxing_machines = equipment.boxing_machines.values[0]
        
        # Initialize the RMI Store
        self.rmi_store = rmi_cell(plant_id, rmi_drums)
        # Initialize the Classifier
        self.classifier = classifier_cell(plant_id)
        # Initialize the PFI Store
        self.pfi_store = pfi_cell(plant_id, pfi_drums)
        # Initialize the PFO Tank
        self.pfo_tank = pfo_cell(plant_id, pfo_tanks)
        # Initialize the PI Store
        self.pi_store = pi_cell(plant_id, pi_drums)
        # Initialize the Packaging Line
        self.packaging_line = packaging_cell(plant_id, bagging_machines, boxing_machines)
        
    def __repr__(self):
        return("{0}".format(self.plant_id))

## 1. Raw Material Inventory (RMI) Store <a class="anchor" id="first-bullet"></a>
Several drums act as an inventory store for raw material. The number of drums and their capacities varies by site. Drums can only hold jelly beans with the same color (many-to-one relationship between drums and jelly beans of a given color). One RMI drum can be released at a time into the classifier.

Rules:
1. Order of release from RMI drums is dictated by workorders
2. If there are two RMI drums with the same color at a given location, start emptying the drum with the lower equipment number first
3. Releasing jelly beans from a drum is only allowed if there are at least 5 pre-finished inventory bins empty and available (the next step, classification, is a continuous process which fills pre-finished inventory bins)

In [126]:
class rmi_cell:
    def __init__(self, plant_id, rmi_drums):
        # Initialize RMI Store using details from provided data
        rmi_il = pd.read_csv('rmi_inventory_level.csv')
        # Subset data on RMI Store Location
        rmi_il = rmi_il[rmi_il['Location Name'] == plant_id]

        
        self.plant_id = plant_id
        self.num_drums = rmi_drums
        self.colors = rmi_il['Color'].dropna()
        
        # Generate a list of RMI Drum object,
        # each with properties as defined by provided data
        self.fill_store(rmi_il)
        
        # Rearrange the list of RMI Drums in accordance with incoming Workorders
        #self.order_drums()
        
    def __repr__(self):
        return(
            "{0} RMI Store \n"
            "Containing {1} Drums \n"
            "With {2} Unique Colors"
            .format(self.plant_id, self.num_drums, len(self.colors))
        )        
    
    def fill_store(self, df):
        print('Filling RMI Drums...')
        self.drums = [
            rmi_drum(
            row['RMI Drum'],
            row['Color'],
            row['Qty in pounds'],
            row['Capacity']
            )
            for index, row in df.iterrows()
        ]
    
    def order_drums(self):
    # Sort list of drums according to work order
    # TODO
        pass
    
    def empty_drums(self):
        empties = [drum for drum in self.drums if np.isnan(drum.contents)]
        return("Empty Drums: {0}".format(empties))
            
            
class rmi_drum:
    def __init__(self, drum_id, color, start_qty, capacity):
        self.id = drum_id
        self.color = color
        self.contents = start_qty
        self.capacity = capacity
        #self.fill()
    
    def __repr__(self):
        return(self.id)
    
    def fill(self, addtl_qty):
        self.contents += self.addtl_qty
        
    def empty(self):
        contents = self.contents
        self.contents = 0
        return(contents)

## 2. Classifier <a class="anchor" id="second-bullet"></a>

The RMI drums contain jelly beans of the same color, but of varying size. RMI drums are emptied one by one into the classifier, which sorts jelly beans into 5 distinct sizes. Percentage split of each jelly bean size for a given color is provided with the problem statement.

In [4]:
class classifier_cell:
    def __init__(self, plant_id):
        print('Initializing Classifier...')
        rates = pd.read_csv('classifier_rate.csv')
        
        self.color = 'Undefined'
        self.plant_id = plant_id
        self.rate = int(rates[rates['Site'] == plant_id]['Processing_Rate'])
        self.id = rates[rates['Site'] == plant_id]['Classifier'].values[0]
        
    
    def __repr__(self):
        return("{0} Classifier\n"
               "ID Number: {1}\n"
               "Processing Color: {2}\n"
               "At Rate: {3} lbs/hr"
               .format(self.plant_id,
                       self.id,
                       self.color,
                       self.rate
               )
        )
    
    def check_pfi_bins(self):
        # TODO
        pass
    
    def classify(self, color, incoming):
        # Return quantity of each size of jelly bean
        print("Classifying {0} {1} Jelly Beans...".format(incoming, color))
        split = pd.read_csv('classifier_split.csv')
        
        self.color = color
        self.split = split[split['Color'] == color]
        
        self.check_pfi_bins()
        split = self.split
        split['qty_out'] = split.apply(lambda x: int(incoming*x.Percentage/100), axis=1)
        time = self.rate/incoming
        print("Classifying Time: {0} Hours".format(time))
        return split[['Size','qty_out']]
        
        
        

## 3. Pre-Finish Inventory (PFI) Store <a class="anchor" id="third-bullet"></a>

Jelly beans sorted by the classifier are stored in PFI drums. Drums are assigned a specific jelly bean size and color to avoid contamination (the PFI drums are reused once the drums are emptied into the Pre-Finish Operation). As mentioned earlier, 5 PFI drums (one for each size) must be empty before an RMI drum can be unloaded (thus starting the classification process).

Rules:
1. The classification process simultaneously splits the jelly beans according to the given ratios directly into the PFI drums
2. Fundamentally any PFI drum can be filled with any size of jelly bean; however, once the assignment is made for a given work order, the PFI drum must only hand jelly beans of that size & color until the work order is completely processed
3. PFI drums can only be filled to 95% capacity
4. PFI drums are only available to be emptied into the Pre-Finish Operation once filling has stopped

In [127]:
class pfi_cell:
    def __init__(self, plant_id, pfi_drums):
        pfi_drums = pd.read_csv('pfi_drum.csv', thousands=',')
        pfi_drums = pfi_drums[pfi_drums['Site'] == plant_id]
        
        self.plant_id = plant_id
        self.num_drums = pfi_drums
        
        self.fill_store(pfi_drums)
        self.assign_drums()
        
    def __repr__(self):
        return(
            "{0} PFI Store \n"
            "Containing {1} Drums"
            .format(self.plant_id, self.num_drums)
        )     
        
    def fill_store(self, df):
        print('Defining PFI Drums...')
        self.drums = [
            pfi_drum(
            row['Drum Number'],
            row['Capacity In pounds']
            )
            for index, row in df.iterrows()
        ]
        
    def assign_drums(self):
        # TODO
        pass
        
class pfi_drum:
    def __init__(self, drum_id, capacity):
        self.id = drum_id
        self.capacity = int(capacity)
        self.contents = 0
        
    def __repr__(self):
        return(self.id)
    
    def assign(self, jb_color, jb_size):
        self.jb_color = jb_color
        self.jb_size = jb_size
    
    def fill(self, amount):
        if (self.contents + amount) <= 0.95*self.capacity:
            self.contents += amount
        else:
            raise ValueError('Amount specified exceeds drum capacity')
        
    def empty(self):
        contents = self.contents
        self.contents = 0
        return(contents)

## 4. Pre-Finish Operation (PFO) <a class="anchor" id="fourth-bullet"></a>

PFI drums are now emptied into a tank which applies flavoring to jelly beans of a given color and size (i.e., the tank can only hold 1 unique color-size-flavor combinations at any given time). The number of tanks, tank capacity, and processing rate are given for each facility as part of the problem statement.

Rules:
1. If there is more than 1 PFO tank, assume the rate for each tank is the same.
2. When flavors are changed, there is a change-over duration of 5 minutes.
3. There is no change-over duration between size changes.

In [13]:
class pfo_cell:
    def __init__(self, plant_id, pfo_tanks):
        self.plant_id = plant_id
        self.num_tanks = pfo_tanks
        
    def __repr__(self):
        # TODO
        pass
    
    def assign_tanks(self):
        # TODO
        pass
        

class pfo_tank:
    def __init__(self, plant_id):
        print("Initializing PFO Tank...")
        self.plant_id = plant_id
        self.color = "Undefined"
        self.rate = np.nan
        self.flavor = "Undefined"
        self.process_time = 0
          
    def __repr__(self):
        return("{0} PFO Tank\n"
              "Processing Color: {1}\n"
              "Processing Flavor: {2}\n"
              "Predicted Processing Rate: {3}"
              .format(
                  self.plant_id,
                  self.color,
                  self.rate,
                  self.flavor
              )
        )
    
    def apply(self, flavor, size, amount):
        
        def process(plant_id, flavor, size, amount):
            # Locate historical rate data for a given location, flavor, and size
            hist_rate = pd.read_csv('pfo_rate.csv') 
            hist_rate = hist_rate[
                (hist_rate.Site == plant_id) 
                & (hist_rate.Flavor == flavor)
                & (hist_rate.Size == size)
            ]['Processing_Rate']

            # Determine sample mean and standard deviation
            s_mean = np.mean(hist_rate)
            s_sd = np.std(hist_rate)

            # Simulate processing rate using random variable
            # Following sample normal distribution
            rand_rate = s_sd*stats.norm.ppf(np.random.random())+s_mean
            self.rate = rand_rate
            self.process_time += amount/rand_rate
        
        self.load(flavor)
        self.flavor = flavor
        process(
            self.plant_id, 
            flavor, 
            size,
            amount
        )
        print(self.process_time)

    def load(self, flavor):
        if (self.flavor != flavor) & (self.flavor != 'Undefined'):
            self.process_time += 5/60
        else:
            pass

## 5. Pack Inventory (PI) Store <a class="anchor" id="fourth-bullet"></a>

Once the PFO is complete, flavored jelly beans are staged in PI drums. Each PI drum contains fully differentiated product: jelly beans grouped by color, size, and flavor. The number of drums and their capacities vary by site and are given within the problem statement.

Rules:
1. Release of jelly beans into the PFO follows a 'FIFO' policy by default (other policies are allowed but must be explicitly specified).
2. Jelly beans flow continuously from the PFI store through the PFO into the PI store. It can be assumed that any PFO can feed any PI drum.
3. To avoid overflow, PI drums are filled to 95%.
4. PI drums can only be emptied into the Packaging operation once they have been disengaged from the PFO.
5. The lowest PI Drum number is filled up first.
6. Jelly bean colors, flavors, and sizes cannot be mixed in a given drum.
7. Only one PI drum can be emptied into the Packaging operation at a time.
8. The quantity of jelly beans released into the Packaging operation will be determined by the Work Orders.

In [93]:
class pi_cell:
    def __init__(self, plant_id):
        
        pi_drums = pd.read_csv('pi_drum.csv', thousands=',')
        pi_drums = pi_drums[pi_drums['Site'] == plant_id]
        
        self.plant_id = plant_id
        
        self.fill_store(pi_drums)
        
    def __repr__(self):
        return("{0} PI Store".format(self.plant_id))
        
    def fill_store(self, df):
        print('Defining PI Drums...')
        self.drums = [
            pi_drum(
            row['Drum Number'],
            row['Capacity']
            )
            for index, row in df.iterrows()
        ]
        
    def assign_drums(self):
        # TODO
        pass
        
class pi_drum:
    def __init__(self, drum_id, capacity):
        self.id = drum_id
        self.capacity = int(capacity)
        self.contents = 0
        self.jb_color = 'Undefined'
        self.jb_size = 'Undefined'
        self.jb_flavor = 'Undefined'
        
    def __repr__(self):
        return(self.id)
    
    def fill(self, jb_color, jb_size, jb_flavor, amount):
        if (self.contents + amount) <= 0.95*self.capacity:
            self.contents += amount
            self.jb_color = jb_color
            self.jb_size = jb_size
            self.jb_flavor = jb_flavor
        elif (self.contents + amount) > 0.95*self.capacity:
            raise ValueError('Amount specified exceeds drum capacity')
        else:
            # Currently unused; add condition for flavor/size/color mismatch
            raise ValueError('Jelly Beans will be contaminated!')

    def empty(self):
        contents = self.contents
        self.contents = 0
        return(contents)
        

## 6. Packaging Operation <a class="anchor" id="sixth-bullet"></a>

PI drums are emptied for packaging either a bag or a box; i.e., a drum can only be emptied to feed a bag or a box.

Rules:
1. The bagging and box lines cannot be run simultaneously.
2. The box line takes precedence over the bagging line.
3. Orders for boxes are fulfilled prior to orders for bags.
4. All excess material is to be stored in bags.

In [ ]:
class packaging_cell:
    def __init__(self, plant_id, bagging_machines, boxing_machines):
        self.plant_id = plant_id
        assign_machines()
        
    def __repr__(self):
        return("{0}".format(plant_id))
    
    def assign_machines(self):
        # TODO
        pass       
    
    
class packing_machine:
    def __init__(self, plant_id):
        self.plant_id = plant_id
        
    def __repr__(self):
        return("{0} unassigned packing machine".format(plant_id))
    

class bagging_machine(packing_machine):
    def __init__(self, plant_id):
        super().__init__(plant_id)
            
    def __repr__(self):
        # TODO
        pass
    
class boxing_machine(packing_machine):
    def __init__(self, plant_id):
        super().__init__(plant_id)
        
    def __repr__(self):
        # TODO
        pass
            

## Workorder Processing

In [ ]:
class workorder_bank:
    def __init__(self, example=True):
        if example:
            self.workorders = pd.read_csv('workorder_example.csv')
        else:
            self.workorders = pd.read_csv('workorders.csv')
            
    def build_bank(self):
        # TODO
        pass
        self.id = workorders

## Testing & Validation

### Facility Initialization

In [107]:
detroit = facility('Detroit, MI')

Building Detroit, MI Facility Details...
Filling RMI Drums...
Initializing Classifier...
Defining PFI Drums...
Initializing PFO Tank...
Defining PI Drums...


In [10]:
columbus = facility('Columbus, OH')

Building Columbus, OH Facility Details...
Filling RMI Drums...
Initializing Classifier...
Defining PFI Drums...


### RMI Store
- Representation of store
- List empty drums
- Representation of drum
- Checking properties of drum

In [ ]:
print(detroit.rmi_store)
print(detroit.rmi_store.empty_drums())
print(detroit.rmi_store.drums[1])
print(detroit.rmi_store.drums[1].contents)

In [ ]:
print(columbus.rmi_store)
print(columbus.rmi_store.empty_drums())
print(columbus.rmi_store.drums[1])
print(columbus.rmi_store.drums[1].capacity)

### Classifier
- Representation of classifier
- Input jelly bean color and amount
- Output processing time
- Output jelly bean size split

In [ ]:
detroit.classifier

In [ ]:
detroit.classifier.classify('Coloring Agent1', 10000)

### PFI Store

- Representation
- List empty drums
- Check drum properties
    - Color assignment
    - Size assignment
    - Capacity
    - Filling limit

In [ ]:
detroit.pfi_store

In [ ]:
print(detroit.pfi_store.drums)
print(detroit.pfi_store.drums[1].capacity)

In [ ]:
detroit.pfi_store.drums[0].assign(jb_color ='Coloring Agent1', jb_size ='S1')
detroit.pfi_store.drums[0].jb_color

In [ ]:
detroit.pfi_store.drums[0].capacity

In [ ]:
detroit.pfi_store.drums[0].fill(9000)
detroit.pfi_store.drums[0].contents

In [ ]:
detroit.pfi_store.drums[0].fill(501)
detroit.pfi_store.drums[0].contents

In [ ]:
detroit.pfi_store.drums[0].empty()
detroit.pfi_store.drums[0].contents

### PFO

- Representation
- Time accrual
- Changeover penalty

In [15]:
detroit.pfo_tank

Detroit, MI PFO Tank
Processing Color: Undefined
Processing Flavor: nan
Predicted Processing Rate: Undefined

In [16]:
detroit.pfo_tank.apply('F1', 'S1', 10000)

7.7040390067822555


In [17]:
detroit.pfo_tank.apply('F1', 'S2', 5000)

11.135808283662591


In [18]:
detroit.pfo_tank.apply('F2', 'S1', 10000)

19.228939424336346


### PI Store

- Representation
- Drums
    - Filling rules
    - Updating state
    - Emptying

In [94]:
detroit.pi_store = pi_store('Detroit, MI')

Defining PI Drums...


In [108]:
detroit.pi_store

Detroit, MI PI Store

In [109]:
detroit.pi_store.drums

[FI Drum 1,
 FI Drum 2,
 FI Drum 3,
 FI Drum 4,
 FI Drum 5,
 FI Drum 6,
 FI Drum 7,
 FI Drum 8]

In [110]:
detroit.pi_store.drums[0].capacity

20000

In [111]:
detroit.pi_store.drums[0].fill('Coloring Agent1', 'S1', 'F1', 10000)

In [112]:
detroit.pi_store.drums[0].fill('Coloring Agent1', 'S1', 'F1', 9000)

In [113]:
detroit.pi_store.drums[0].fill('Coloring Agent1', 'S1', 'F1', 1)

ValueError: Amount specified exceeds drum capacity

In [114]:
detroit.pi_store.drums[0].contents

19000

In [115]:
detroit.pi_store.drums[0].jb_color

'Coloring Agent1'

In [116]:
detroit.pi_store.drums[0].jb_size

'S1'

In [117]:
detroit.pi_store.drums[0].empty()

19000

In [118]:
detroit.pi_store.drums[0].contents

0

### Packaging

In [ ]:
# Attempt at writing super/sub class for drums
class drum:
     def __init__(self, drum_id, capacity):
        self.id = drum_id
        self.capacity = int(capacity)
        self.contents = 0
        self.jb_color = 'Undefined'
        self.jb_size = 'Undefined'
        self.jb_flavor = 'Undefined'
        
    def __repr__(self):
        return(self.id)

In [ ]:
sample_mean = np.mean(hist_rate)
sample_sd = np.std(hist_rate)

rand_num = np.random.random()
rand_rate = sample_sd*stats.norm.ppf(rand_num)+sample_mean

fig, ax1 = plt.subplots(constrained_layout=True)
x = np.linspace(sample_mean - 4*sample_sd, sample_mean + 4*sample_sd, 1000)
y1 = hist_rate
y2 = stats.norm.pdf(x, sample_mean, sample_sd)
ax1.plot(x, y2, color='r')
ax2 = ax1.twinx()
ax2.hist(y1, bins=25, alpha=0.5)
ax2.axvline(x=rand_rate, color='black')
plt.show();

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)
y3 = stats.norm.cdf(x, sample_mean, sample_sd)
ax1.plot(x, y2, color='r')
ax2 = ax1.twinx()
ax2.plot(x, y3, color='c')
plt.show();